In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import math
import cmath
import sys
import IPython.display as ipd

from sklearn import preprocessing
from sklearn.datasets import make_regression
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from scipy import signal

In [2]:
"""
Open the training data.
"""
training_data_filepath = 'F:/ZaknafeinII_Backup_02-02-22/daea/training_data_generation/id16/output/stft_training_data_dict.pickle'

with open(training_data_filepath, 'rb') as f:
    data_dict = pickle.load(f)
    
for key in data_dict:
    print(key, 'is of shape', data_dict.get(key).shape)

train_input is of shape (1048, 129, 691)
val_input is of shape (225, 129, 691)
test_input is of shape (224, 129, 691)
train_targets is of shape (1048, 129, 691)
val_targets is of shape (225, 129, 691)
test_targets is of shape (224, 129, 691)
train_noise is of shape (1048, 129, 691)
val_noise is of shape (225, 129, 691)
test_noise is of shape (224, 129, 691)


In [3]:
"""
We need to set up the train array, "Sy", and the target array, "R".
"""
train_input = data_dict.get('train_input')
stft_clean = data_dict.get('train_targets')
stft_noise = data_dict.get('train_noise')

# R
train_target = []

for index in range(0, len(train_input)):
    Px = np.square(stft_clean[index])
    Pn = np.square(stft_noise[index])
    R = Px / sum([Px, Pn])
    
    train_target.append(R)
    
train_target = np.array(train_target)

print('train_input.shape:', train_input.shape)
print('train_target.shape:', train_target.shape)

train_input = np.swapaxes(train_input, 1, 2)
train_target = np.swapaxes(train_target, 1, 2)

print('train_input.shape:', train_input.shape)
print('train_target.shape:', train_target.shape)

# train_input = train_input.reshape(train_input.shape[0]*train_input.shape[1], train_input.shape[2])
# train_target = train_target.reshape(train_target.shape[0]*train_target.shape[1], train_target.shape[2])

# print('train_input.shape:', train_input.shape)
# print('train_target.shape:', train_target.shape)



train_input = train_input.reshape(1048, 1, 691, 129)
train_target = train_target.reshape(1048, 1, 691, 129)

print('train_input.shape:', train_input.shape)
print('train_target.shape:', train_target.shape)

train_input = train_input[:, :, 0:690, :]
train_target = train_target[:, :, 0:690, :]

print('train_input.shape:', train_input.shape)
print('train_target.shape:', train_target.shape)

train_input.shape: (1048, 129, 691)
train_target.shape: (1048, 129, 691)
train_input.shape: (1048, 691, 129)
train_target.shape: (1048, 691, 129)
train_input.shape: (1048, 1, 691, 129)
train_target.shape: (1048, 1, 691, 129)
train_input.shape: (1048, 1, 690, 129)
train_target.shape: (1048, 1, 690, 129)


In [4]:
# fs = 22050
# input_ = train_input[0][:,0:100]
# _, xrec = signal.istft(input_, fs)
# print('input dim:', input_.shape)
# print('output dim:', xrec.shape)
# ipd.Audio(xrec, rate=fs)

# print()
# input_ = train_input[0][:,0]
# _, xrec = signal.istft(input_, fs)
# print('input dim:', input_.shape)
# print('output dim:', xrec.shape)
# ipd.Audio(xrec, rate=fs)

# counter = 0
# for i in train_input[0]:
#     if counter < 1:
#         print(len(i))
#         counter = counter + 1
#     else:
#         break

In [5]:
# # do we need the last axis of the Zxx? we should see if it can be reconstructed without it
from keras.layers import Dense, Dropout, Activation, Flatten, MaxPooling1D, Conv1D, LSTM, Conv2D


# NN_model = Sequential()

# # The Input Layer :
# NN_model.add(Dense(129, kernel_initializer='normal', input_dim = 129, activation='relu'))

# # The Hidden Layers :
# NN_model.add(Dense(172, kernel_initializer='normal', activation='relu'))
# NN_model.add(Dense(129, kernel_initializer='normal', activation='relu'))

# # The Output Layer :
# # changing the output dimensions from 1 to 64
# NN_model.add(Dense(129, kernel_initializer='normal', activation='linear'))

# # Compile the network :
# NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
# NN_model.summary()

model = Sequential()

# https://stackoverflow.com/questions/61816859/how-to-create-keras-conv2d-layer-on-grayscale-image-set
# model.add(Conv1D(filters=258, kernel_size=5, padding='same', activation='relu', input_shape=(None, 129)))
model.add(Conv2D(64, (3,3), activation="relu", padding='same', input_shape=(1, 690, 129)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
# model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
# model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (1, 1), activation='relu', padding='same'))


# model.add(MaxPooling1D(pool_size=4))

# model.add(LSTM(64))

model.add(Dense(129, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')#, metrics=[metric])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 690, 64)        74368     
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 690, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 690, 256)       295168    
                                                                 
 conv2d_3 (Conv2D)           (None, 1, 690, 128)       32896     
                                                                 
 dense (Dense)               (None, 1, 690, 129)       16641     
                                                                 
Total params: 492,929
Trainable params: 492,929
Non-trainable params: 0
_________________________________________________________________


In [6]:
"""
Only save the best form of the model throughout the training process.
https://towardsdatascience.com/deep-neural-networks-for-regression-problems-81321897ca33

Notes:
- look into pruning during training
- explore this link: https://towardsdatascience.com/how-to-build-a-neural-network-for-voice-classification-5e2810fe1efa
  - has dropout layers
  - has graph to show performance over time during training
  - has early stopping
  - has layers to study
- train with GPU somehow
- better data preprocessing
- From librosa concerning MFCC generation: "If multi-channel audio input y is provided, the MFCC calculation 
  will depend on the peak loudness (in decibels) across all channels. The result may differ from independent 
  MFCC calculation of each channel."
"""
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.python.keras import backend as K

# adjust values to your needs
config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} )
sess = tf.compat.v1.Session(config=config) 
K.set_session(sess)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


# checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
# checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
# callbacks_list = [checkpoint]

# early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')

model.fit(
          train_input, 
          train_target, 
          epochs=64, 
          batch_size=16, 
          validation_split = 0.2, 
#           callbacks=callbacks_list, 
          shuffle=True
          )

Num GPUs Available:  0
Epoch 1/64
53/53 [==============================] - 15s 269ms/step - loss: 0.5519 - val_loss: 0.3913
Epoch 2/64
53/53 [==============================] - ETA: 0s - loss: 0.4127

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential/Cast' defined at (most recent call last):
    File "C:\Users\ZaknafeinII\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\ZaknafeinII\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\ZaknafeinII\anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\ZaknafeinII\anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\ZaknafeinII\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\ZaknafeinII\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\ZaknafeinII\anaconda3\lib\asyncio\base_events.py", line 596, in 

In [ ]:
test_input = data_dict.get('test_input')
# test_target = data_dict.get('test_targets')
test_input = np.swapaxes(test_input, 1, 2)
# test_target = np.swapaxes(test_target, 1, 2)
# test_input = test_input.reshape(test_input.shape[0]*test_input.shape[1], test_input.shape[2])
# test_target = test_target.reshape(test_target.shape[0]*test_target.shape[1], test_target.shape[2])
# print('test_input.shape:', test_input[0].shape)
# print('test_target.shape:', test_target.shape)


In [ ]:
print(test_input[0:2].shape)
predictions = np.array(model.predict(test_input[0:2]))
print(predictions.shape)
# y = imask_test[0]
# yhat = NN_model.predict(x)

In [ ]:
print(predictions.shape)

In [ ]:
xhat = predictions[1] * test_input[1]

In [ ]:
_, xrec = signal.istft(xhat, 22050)
print(xrec.shape)
ipd.Audio(xrec, rate=22050)